<a href="https://colab.research.google.com/github/demax44/Capstone/blob/master/Capstone_Neighborhood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

The_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(The_url).text
Postal_Data=BeautifulSoup(r,'lxml')

New_namescol=['Postal code','Borough','Neighborhood']
Canada_Code=pd.DataFrame(columns=New_namescol)


CTamont=Postal_Data.find('div',class_= "mw-parser-output")
table = CTamont.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    Canada_Code = Canada_Code.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)



Canada_Code = Canada_Code[Canada_Code.Borough!='Not assigned']
Canada_Code = Canada_Code[Canada_Code.Borough!= 0]
Canada_Code.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,Canada_Code.shape[0]):
    if Canada_Code.iloc[i][2] == 'Not assigned':
        Canada_Code.iloc[i][2] = Canada_Code.iloc[i][1]
        i = i+1
                                 
df1 = Canada_Code.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp= df1.groupby(['Postalcode','Borough'])
DataFrame3 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
DataFrame3.rename(columns={'Postalcode':'Postal Code'}, inplace=True)
DataFrame3.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


In [0]:
Canadageo=pd.read_csv('/content/Geospatial_Coordinates.csv').set_index('Postal Code')

In [45]:
Canadageo

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
...,...,...
M9N,43.706876,-79.518188
M9P,43.696319,-79.532242
M9R,43.688905,-79.554724


Merge the Data into one

In [50]:
DataFrame3 = pd.merge(Canadageo, DataFrame3, on= 'Postal Code')
DataFrame3.head()

,Postal Code,Latitude,Longitude,Borough,Neighborhood
